In [1]:
import pandas as pd
import geopandas as gpd

import shared_utils
from utils import *
import intake
import gcsfs

import calitp
from calitp.tables import tbl
from siuba import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
coverage_gdf = gpd.read_parquet(f'{GCS_FILE_PATH}block_accessibility_metrics.parquet')

In [3]:
coverage_gdf

,county,tract,block,geo_id,geometry,block_pop,area,rt_accessible,rt,accessible,static
0,17,30706,3012,60170307063012,"POLYGON ((-93795.217 73341.696, -93626.457 734...",363,8.358171e+04,False,False,False,False
1,17,30601,3006,60170306013006,"POLYGON ((-88050.178 97425.705, -88018.654 977...",64,9.336402e+05,False,False,False,False
2,65,42628,4008,60650426284008,"POLYGON ((257831.965 -463783.472, 258008.040 -...",103,1.598696e+04,False,False,False,False
3,65,42624,2046,60650426242046,"POLYGON ((267587.010 -452823.837, 267988.443 -...",0,1.292617e+06,False,False,False,False
4,65,42717,3007,60650427173007,"POLYGON ((255572.531 -476570.440, 255679.050 -...",45,2.891209e+04,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
511802,37,570204,1000,60375702041000,"POLYGON ((170949.000 -458449.414, 171055.512 -...",115,2.137071e+04,False,True,False,True
511803,37,554511,1007,60375545111007,"POLYGON ((181539.157 -457524.885, 181846.402 -...",0,1.290997e+05,False,False,False,False
511804,37,603102,1009,60376031021009,"POLYGON ((157994.818 -458021.668, 158074.190 -...",30,1.018442e+04,False,False,False,True
511805,37,573700,1014,60375737001014,"POLYGON ((175275.405 -463832.107, 175352.230 -...",65,1.978373e+04,False,True,False,True


In [13]:
popup_dict = {
    "rt_accessible": "Accessible Trip and Stop, RT Feed Available",
    "rt": "RT Feed Available",
    "accessible": "Accessible Trip and Stop",
    "static": "GTFS Static Present",
}

In [5]:
import branca

In [9]:
from shared_utils import calitp_color_palette as cp

In [10]:
cp.CALITP_CATEGORY_BRIGHT_COLORS

['#2EA8CE', '#EB9F3C', '#F4D837', '#51BF9D', '#8CBCCB', '#9487C0']

In [20]:
LAYERS_DICT = {
    "RT Accessible": {"df": coverage_gdf >> filter(_.rt_accessible),
        "plot_col": "rt_accessible",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[:1],
    },
    "RT": {"df": coverage_gdf >> filter(_.rt),
        "plot_col": "rt",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[1:2],
    },
    "Accessible": {"df": coverage_gdf >> filter(_.accessible),
        "plot_col": "accessible",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[2:3],
    },
    "GTFS Static": {"df": coverage_gdf >> filter(_.static),
        "plot_col": "static",
        "popup_dict": popup_dict, 
        "tooltip_dict": popup_dict,
        "colorscale": lambda x: cp.CALITP_CATEGORY_BRIGHT_COLORS[3:4],
    }
}

In [ ]:
# shared_utils.map_utils.make_folium_multiple_layers_map?

In [ ]:
fig = shared_utils.map_utils.make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = 700, fig_height = 700, 
        zoom=9, 
        centroid = [36.5, -120], 
        title=f"Statewide Feed Coverage",
        # legend_dict = LEGEND_DICT
    )

In [ ]:
fig.show()